In [ ]:
#首先RNN形式有很多，但是使用较多一般是LSTM或者GRU，因为lstm是原版基础
#LSTM与GRU本质上是同源，即GRU也是基于LSTM改进的

#LSTM有三个门：1、遗忘门 2、输入门 3、输出门
#相比LSTM：
#   1、GRU少了一个门，即将LSTM中遗忘门与输入门合并成一个门
#   2、合并了数据单元状态和隐藏状态(即lstm会有两个方向的输出：纵向的特征信息输出和横向的状态输出，而GRU合并后只有一个了：纵向特征信息)
#   3、整体结构更简单了，原本lstm第2个门和第3个门都各自会有一个sigmoid层，而GRU中则将第二个门的sigmoid接上一个"1-"之后给了第三个门做sigmoid使用

#tf关于lstm的API接口
tf.contrib.rnn.BasicLSTMCell(hidden_units, forget_bias=1.0, state_is_tuple=True)
tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
#这两个不同接口，但是指向的是相同的tf的源代码，所以用哪个都可以
#hidden_units即表示内部维度数，也就是隐藏神经元的维度数，也就是输出的维度数（有点类似于CNN中卷积核个数的作用一样）
#forget_bias：遗忘门的偏置，默认为1，一般也不用去改
#state_is_tuple：因为lstm有两个方向上的输出，故而如果为True，则以一个二元组的方式返回，若为True，则按列进行concatenate之后再输出

#tf关于GRU的API接口
tf.contrib.rnn.GRUCell(hidden_dim)
#相比lstm而言，GRU第一没有forget_bias，因为GRU将lstm的第一个遗忘门和第二个输入门合并了。第二也没有state_is_tuple，因为GRU只有一个纵向输出
#而这里的hidden_dim则是和lstm一样，都是属于内部隐藏神经元个数，也就是控制着输出维度

#因为一般而言，为防止过拟合都会有一个池化或者dropout的压缩操作。
#故而一般定义了一层rnn后，会跟上一个dropout
tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=self.keep_prob) #若是1.0的话，则有和没有则没有太大区别了，一般设0.8

#正常情况下，多层rnn会比一层rnn效果好，因为获取到的信息更丰富。但是不宜太多，因为会遗忘掉。一般两三层足以。
#而要多层RNN，则需要将所有rnn层拼接起来，一般会使用MultiRNNCell来拼接，当然，以下两种方式完全一样
tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)
tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_fw, state_is_tuple=True)
#这里面的参数cells对应的值需要是一个列表形式

#将rnn拼接完了之后，也就是准备好了rnn的网络层次后，需要放到rnn网络中
#放到rnn网络中，整体有两类：一类是单向的，另一类是双向的，一般而言，双向的会比单向的要好。但是在seq2seq(attention)时测试的时候只能是单向的。
#这两大类都有两种类型网络，即动态rnn和静态rnn。静态rnn需要保证所有批次的所有样本长度一致，而动态rnn则只要保证批次内样本长度一致即可。
outputs, _ = tf.nn.dynamic_rnn(cell=rnn_cell, sequence_length=None,inputs=embedding_inputs, dtype=tf.float32)
#注意：'outputs' is a tensor of shape [batch_size, max_time, cell_state_size]   #这里的max_time即为批次内样本长度，cell_state_size即为维度数
#return的会有两个：(outputs, final_state)，横向上的状态输出和纵向上的特征信息输出，一般我们只需要第0个
#虽然这个是单向的，但这个是动态的，故而可以设置sequence_length，默认为None，即默认所有批次长度一样，但是可以针对每批次进行调整
outputs, _, _ = tf.nn.static_bidirectional_rnn(lstm_fw_cell_m, lstm_bw_cell_m, x, dtype=tf.float32)
#注意：outputs is a length `T` list of outputs (one for each input)
#这个是双向的，故而return的会有三个：(outputs, output_state_fw, output_state_bw)，即纵向上的最终特征信息，以及前向和后向上个字的横向状态输出
#当然，如果是双向的，则需要配套定义后向的rnn_cells,与前向的需要等大小

#关于bidirectional_dynamic_rnn的，需要看下源码解释，写的时候需要注意一下
#最后可能需要先对结果输出进行tf.concat(outputs, 2)

In [ ]:
#多层RNN示例：
def lstm_cell():   # lstm核
    return tf.contrib.rnn.BasicLSTMCell(hidden_dim, forget_bias=1.0, state_is_tuple=True)

def gru_cell():  # gru核
    return tf.contrib.rnn.GRUCell(hidden_dim)

def dropout(): # 为每一个rnn核后面加一个dropout层
    if (self.config.rnn == 'lstm'):
        cell = lstm_cell()
    else:
        cell = gru_cell()
    return tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=self.keep_prob)

In [ ]:
#单向动态RNN
with tf.name_scope("rnn"):
    cells = [dropout() for _ in range(self.config.num_layers)]  
    rnn_cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)

    outputs, _ = tf.nn.dynamic_rnn(cell=rnn_cell, inputs=embedding_inputs, dtype=tf.float32)

In [ ]:
#双向静态RNN
with tf.name_scope("fw"+scope),tf.variable_scope("fw"+scope):
    fw_cells = [dropout() for _ in range(self.config.num_layers)]  
    fw_rnn_cells = tf.contrib.rnn.MultiRNNCell(fw_cells, state_is_tuple=True)

with tf.name_scope("bw"+scope),tf.variable_scope("bw"+scope):
    bw_cells = [dropout() for _ in range(self.config.num_layers)]  
    bw_rnn_cells = tf.contrib.rnn.MultiRNNCell(bw_cells, state_is_tuple=True)

with tf.name_scope("rnn"):
    outputs, _, _ = tf.nn.static_bidirectional_rnn(lstm_fw_cell_m, lstm_bw_cell_m, x, dtype=tf.float32)    

In [ ]:
#正常而言，outputs的大小是和输入的x的维度一样的
#比如input的x正常维度是batch_size * 样本长度 * 每个字/词向量的维度数
#那么在传入tf.nn.dynamic_rnn 或者tf.nn.static_bidirectional_rnn 之后，其输出维度也会是batch_size * 样本长度 * 每个字/词向量的维度数

#但如果x在传入rnn网络之前做了转置操作，比如x = tf.unstack(tf.transpose(x, perm=[1, 0, 2]))
#变成了 样本长度 * batch_size * 每个字/词向量的维度数
#那么最后获取到的也会是 样本长度 * batch_size * 每个字/词向量的维度数

#又由于lstm一般最后只看最后一个字/词向量的特征信息输出，故而对于outputs会做进一步取值操作，即
#最终总是要获取到最后一个字 * batch_size * 维度 作为后续操作的特征信息
#故而若是dynamic_rnn，因为返回来的是[batch_size, max_time, cell_state_size] ，故而
last_output = outputs[:, -1, :] # 即取所有批次，最后一个字/词的所有维度向量

#当然，如果是使用的static_bidirectional_rnn，也就是输出的会是outputs is a length `T` list of outputs (one for each input)
#若不对输入前的x进行转置的话，那么要获取列表每项元素，然后对每项进行取[-1,:]
#若在传入网络前对输入x做了一下转置的话，那么取的话也能简单一些
last_output = outputs[-1]  #最后一个元素，即所有批次的最后字

In [ ]:
#拿到最终的last_output之后，则根据不同的目的进行不同的处理操作了。
#比如做句子相似度，则需要去计算欧式距离，计算对比损失函数。
#           当然，直接拿outputs去计算可能最后准确度会低一点
#           那么对last_output做一次1*1的卷积操作和3*3的卷积操作，可能准确度会更高些。因为既能获取更丰富的单字信息，也能获取多字的特征信息。
#比如做文本分类，则需要再加一个全连接层(主要是为了获取到对应结果维度)，然后再进行softMax，求最大可能。
#           当然，在全连接层前面再加一个全连接层+relu激活，来丰富特征信息也不无不可，可以试试效果。